In [25]:
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [26]:
dataframe = pd.read_csv('data/lab1/krkopt.csv')
dataframe

,wh_king_file,wh_king_rank,wh_rook_file,wh_rook_rank,bl_king_file,bl_king_rank,res
0,a,1,b,3,c,2,draw
1,a,1,c,1,c,2,draw
2,a,1,c,1,d,1,draw
3,a,1,c,1,d,2,draw
4,a,1,c,2,c,1,draw
...,...,...,...,...,...,...,...
28051,b,1,g,7,e,5,sixteen
28052,b,1,g,7,e,6,sixteen
28053,b,1,g,7,e,7,sixteen
28054,b,1,g,7,f,5,sixteen


In [27]:
dataframe = dataframe.replace({
    "draw": -1,
    "zero": 0,
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
    "ten": 10,
    "eleven": 11,
    "twelve": 12,
    "thirteen": 13,
    "fourteen": 14,
    "fifteen": 15,
    "sixteen": 16,
})
dataframe

,wh_king_file,wh_king_rank,wh_rook_file,wh_rook_rank,bl_king_file,bl_king_rank,res
0,a,1,b,3,c,2,-1
1,a,1,c,1,c,2,-1
2,a,1,c,1,d,1,-1
3,a,1,c,1,d,2,-1
4,a,1,c,2,c,1,-1
...,...,...,...,...,...,...,...
28051,b,1,g,7,e,5,16
28052,b,1,g,7,e,6,16
28053,b,1,g,7,e,7,16
28054,b,1,g,7,f,5,16


In [28]:
feature_columns = []

for column in ['wh_king_rank', 'wh_rook_rank', 'bl_king_rank']:
    feature_columns.append(feature_column.numeric_column(column))

for column in ['wh_king_file', 'wh_rook_file', 'bl_king_file']:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
      column, dataframe[column].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)

feature_columns

[NumericColumn(key='wh_king_rank', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='wh_rook_rank', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='bl_king_rank', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='wh_king_file', vocabulary_list=('a', 'b', 'c', 'd'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='wh_rook_file', vocabulary_list=('b', 'c', 'd', 'e', 'f', 'g', 'h', 'a'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='bl_king_file', vocabulary_list=('c', 'd', 'e', 'f', 'g', 'h', 'a', 'b'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [29]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.25)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

16833 train examples
5611 validation examples
5612 test examples


In [30]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [31]:
def dataframe_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('res')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

train_ds = dataframe_to_dataset(train)
val_ds = dataframe_to_dataset(val)
test_ds = dataframe_to_dataset(test, shuffle=False)

In [39]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(64, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(18)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
503/527 [===========================>..] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0032WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'wh_king_file': <tf.Tensor 'IteratorGetNext:2' shape=(None,) dtype=string>, 'wh_king_rank': <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>, 'wh_rook_file': <tf.Tensor 'IteratorGetNext:4' shape=(None,) dtype=string>, 'wh_rook_rank': <tf.Tensor 'IteratorGetNext:5' shape=(None,) dtype=int64>, 'bl_king_file': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=string>, 'bl_king_rank': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>}
Consider rewriting this model with the Functional API.
527/527 [==============================] - 2s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0031 - val_loss: 0.0000e+00 - val_accuracy: 0.0012
Epoch 2/10
527/5

In [33]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

176/176 [==============================] - 0s 1ms/step - loss: -123194136.0000 - accuracy: 0.0032
Accuracy 0.0032074127811938524
